## Conexão e extração dos dados do Twitter

In [1]:
!pip install textblob

  Running setup.py install for nltk: started
    Running setup.py install for nltk: finished with status 'done'


You are using pip version 19.0.3, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
!pip install tweepy

You are using pip version 19.0.3, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
!pip install numpy

You are using pip version 19.0.3, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [6]:
!pip install pandas

You are using pip version 19.0.3, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [8]:
!pip install sklearn

  Running setup.py install for sklearn: started
    Running setup.py install for sklearn: finished with status 'done'


You are using pip version 19.0.3, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [9]:
import tweepy
import numpy as np
from textblob import TextBlob as tb

from nltk import word_tokenize
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

In [10]:
# Variaveis de acesso, keys cedidas pela api do twitter
consumer_key = "D7WbZKB1D8sI6SG5YSJaoUqFw"
consumer_secret = "Ce2wShKqGIeMgHrmugmpNRK1AAGNXik1imJ47sLJnF4dhFKLqT"
access_token = "758066017742512128-UaPf9IJfaJ957JO5nb4F8TNVa7PBH3p"
access_token_secret = "amPLoPVU6hyOXUyNVRRVrzZPU6zP0lT8Mnt7eLxhbFzdF"

In [11]:
# Realizando a autenticação
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [12]:
# Realizando a conexão com a API
api = tweepy.API(auth)

In [24]:
# Variável que irá armazenar todos os Tweets com a palavra escolhida na função search da API
#public_tweets = api.search('Xiaomi')
public_tweets = api.search('xiaomi mi9', 
                           lang='pt', 
                           count=100,
                          result_type='mixed'
                          )

In [25]:
def print_status(status_list):
    for status in status_list:
        print(status.created_at, ':', status.text+'\n')

In [26]:
print_status(public_tweets)

2019-09-28 16:44:07 : @Miltonneves Tranquila, tranquila...kkkkk...o bicho querendo devorar a criança e a retardada... tranquila

2019-09-28 16:39:02 : @ThiagoCiopek To gostando pra caramba da Xiaomi ^^ peguei um Mi9 e sinceramente ótimo!

2019-09-28 05:48:10 : esse xiaomi mi9 lite tá MUITO foda

2019-09-28 05:16:43 : Comprei o Xiaomi Mi9, negas. O esmurrou da China no USA é real.

2019-09-28 00:19:08 : @Ze36Machado Telemóvel é trocar do Xiaomi Mi6 para o Mi9 porque não sou gajo de gastar 800 euros num telemóvel só p… https://t.co/ikunWvSyMA

2019-09-27 20:28:59 : (2 em 1) Cabo Micro USB e Type-C 3A BlitzWolf BW-MT3 (3FT/91cm) #Banggood

R$ 12.97 (USD $2.99)
O Preço é Igual ao… https://t.co/0tCDG3ms0K

2019-09-27 19:13:10 : @JonBeltrami Pô mano, tem o Xiaomi mi9 tbm, acho que vale a pena
Se tiver um pouco mais de grana, talvez um iPhone… https://t.co/hvI7I62HkK

2019-09-27 18:51:58 : @iamsheylla a do mi9 se é melhor sim
se vc usar a gcam entao

https://t.co/TPC8gXfDLM

tem uns exemplos 

In [28]:
print(len(public_tweets))

93


## Manipulação dos dados para gerar um data frame com pandas

In [8]:
# Imprimir todos os tweets
# Cria a lista com algumas informações dos tweets extraidos

def cria_lista_tweets(public_tweets):
    tweets = [] # Lista vazia para armazenar scores
    for tweet in public_tweets:
        #print(tweet)
        print(tweet.id)
        print(tweet.created_at)
        print(tweet.text)
        print()

        lista_aux = []
        lista_aux.append(tweet.id)
        lista_aux.append(tweet.created_at)
        lista_aux.append(tweet.text)

        tweets.append(lista_aux)

    #print(tweets)
    return tweets

In [9]:
tweets = cria_lista_tweets(public_tweets)

1171920866181025792
2019-09-11 22:58:02
RT @cryptolobopt: Entre um Xiaomi Mi9 e um iPhone 8, o Xiaomi leva a melhor e ainda é mais barato cerca de 120€. Parece-me incrível que a A…

1171909987976785922
2019-09-11 22:14:48
RT @compiws: serio que tem gente falando que xiaomi vai copiar as 3 cameras''fogão de 3 bocas'' do iphone 11,vei xiaomi fez isso antes que…

1171894182606245889
2019-09-11 21:12:00
RT @cryptolobopt: Entre um Xiaomi Mi9 e um iPhone 8, o Xiaomi leva a melhor e ainda é mais barato cerca de 120€. Parece-me incrível que a A…

1171886123435679745
2019-09-11 20:39:59
Meu amigo -Mano eu já conhecia o xaomi, mas sempre tive receio de comprar, dei 4k no IPhone X.
Hoje chegou o Xiaomi… https://t.co/9uMtQyjiuX

1171876391819927553
2019-09-11 20:01:18
RT @cryptolobopt: Entre um Xiaomi Mi9 e um iPhone 8, o Xiaomi leva a melhor e ainda é mais barato cerca de 120€. Parece-me incrível que a A…

1171875785621299202
2019-09-11 19:58:54
RT @cryptolobopt: Entre um Xiaomi Mi9 e um iPhone 8

In [23]:
def cria_arquivo_csv(lista):
    arquivo = open('tweets_xiaomi.csv', 'w', encoding='utf-8')
  
    arquivo.write('id,created_at,text' + '\n')
    
    for tweet in lista:
        #print(tweet)
        for pos in range(len(tweet)):
            if pos == len(tweet)-1:
                arquivo.write('"' + str(tweet[pos]) + '"')
            else:
                arquivo.write(str(tweet[pos]))
                arquivo.write(',')
                
        arquivo.write('\n')
                
        
    arquivo.close()

In [24]:
cria_arquivo_csv(tweets)

In [25]:
dataset = pd.read_csv('tweets_xiaomi.csv', encoding='utf-8')

In [26]:
type(dataset)

pandas.core.frame.DataFrame

In [27]:
dataset.head(10)

,id,created_at,text
0,1171920866181025792,2019-09-11 22:58:02,RT @cryptolobopt: Entre um Xiaomi Mi9 e um iPh...
1,1171909987976785922,2019-09-11 22:14:48,RT @compiws: serio que tem gente falando que x...
2,1171894182606245889,2019-09-11 21:12:00,RT @cryptolobopt: Entre um Xiaomi Mi9 e um iPh...
3,1171886123435679745,2019-09-11 20:39:59,"Meu amigo -Mano eu já conhecia o xaomi, mas se..."
4,1171876391819927553,2019-09-11 20:01:18,RT @cryptolobopt: Entre um Xiaomi Mi9 e um iPh...
5,1171875785621299202,2019-09-11 19:58:54,RT @cryptolobopt: Entre um Xiaomi Mi9 e um iPh...
6,1171844923265564672,2019-09-11 17:56:16,@BeyondTheWings_ Não vai ter eu ganhando um Xi...
7,1171840867197431810,2019-09-11 17:40:09,@dsouza_55 Faz essa merda não mano. 6s totalme...
8,1171838224718868480,2019-09-11 17:29:39,Quem quiser me da um xiaomi mi9 de presente to...
9,1171819246147506176,2019-09-11 16:14:14,"@kruda1020 Queria um Mi9 tbm, q Xiaomi é boa\r..."


In [29]:
# Contabiliza a quantidade de elementos do data frame
dataset.count()

id            15
created_at    15
text          15
dtype: int64

## Pre-processamento dos Dados

In [30]:
# Retorna o nome das colunas do data frame
dataset.columns

Index(['id', 'created_at', 'text'], dtype='object')

In [31]:
# Remove textos duplicados
dataset.drop_duplicates(['text'], inplace=True)

In [32]:
# Contabiliza a quantidade de elementos do data frame, depois de remover os dados duplicados
dataset.text.count()

10

In [33]:
import nltk
nltk.download('stopwords')
nltk.download('rslp')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MaluF\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\MaluF\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [34]:
def RemoveStopWords(instancia):
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

In [35]:
def Stemming(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = []
    for w in instancia.split():
        palavras.append(stemmer.stem(w))
    return (" ".join(palavras))

In [36]:
def Limpeza_dados(instancia):
    # remove links, pontos, virgulas,ponto e virgulas dos tweets
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    return (instancia)